# Driver Drowsiness Detection
This project detects driver drowsiness using a Convolutional Neural Network (CNN) built on top of the InceptionV3 model. It classifies eye states (open/closed) using image data, trained on the MRL Eye Dataset.

Model was trained on Kaggle, and real-time detection is handled separately via a Python script.

## 📁 Dataset
The dataset used is the **MRL Eye Dataset**, containing labeled images of eyes (open and closed). Data is split into training, validation, and testing subsets.

In [ ]:
# Directory setup and imports
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

## 🧪 Image Preprocessing
We use ImageDataGenerator for preprocessing and augmentation.

In [ ]:
IMG_SIZE = 80
BATCH_SIZE = 8

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)

# Replace with your dataset path
train_data = train_datagen.flow_from_directory(
    'path_to_train_folder',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training')

validation_data = train_datagen.flow_from_directory(
    'path_to_train_folder',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation')

## 🧠 Model Architecture (InceptionV3)
Transfer learning is used with InceptionV3 (pre-trained on ImageNet). Custom layers are added on top.

In [ ]:
base_model = InceptionV3(
    include_top=False,
    weights='imagenet',
    input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3))
)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

## ⚙️ Compilation and Training

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3)

history = model.fit(
    train_data,
    validation_data=validation_data,
    epochs=30,
    callbacks=[checkpoint, early_stopping, reduce_lr],
    verbose=1
)

## ✅ Evaluation
Evaluate performance on training and validation sets.

In [ ]:
acc_train, loss_train = model.evaluate(train_data)
acc_val, loss_val = model.evaluate(validation_data)
print(f'Training Accuracy: {acc_train}, Loss: {loss_train}')
print(f'Validation Accuracy: {acc_val}, Loss: {loss_val}')

## 🎥 Real-Time Drowsiness Detection (Separate Script)
The real-time webcam-based drowsiness detection is implemented in a **separate Python script** (`webcam_detection.py`).

To run the live detection system:
```bash
python webcam_detection.py
```

Make sure the following files are present:
- `best_model.h5` (trained model)
- `alarm.wav` (alarm sound)
- Twilio credentials & ngrok URL configured inside the script